In [147]:
from bayesian_optimization import BayesianOptimization
import pandas as pd 
from botorch.test_functions import Hartmann
from varstool import DVARS
import seaborn as sns
import matplotlib.pyplot as plt

# Variables that determine the size of the dataset
N_INITIAL = 50
EPOCHS = 10
BATCH_SIZE = 5
DIM = 6
LOWER = 0
UPPER = 1

optimizer = BayesianOptimization(fun=Hartmann(dim=DIM,negate=True), 
                                 batch_size=BATCH_SIZE, 
                                 dim=DIM, 
                                 epochs=EPOCHS, 
                                 n_init=N_INITIAL, 
                                 lower_bound=LOWER,
                                 upper_bound=UPPER,
                                 seed=3,
                                 acqf_type='qUCB')

x_max, y_max = optimizer.run()
data = optimizer.get_data()
full = optimizer.format(data, dim=DIM, n_init=N_INITIAL, batch_size=BATCH_SIZE, epochs=EPOCHS)
display(full)

vars_data = full.drop(columns=['Batch'])

display(vars_data)
vars_data.to_csv('vars_data.csv', index=False)
h = 0.3
experiment = DVARS(data_file = 'vars_data.csv',
                   outvarname = 'y',
                   ivars_range = h,
                   phi0=1,
                   phi_max=1e6,
                   tol = 1e-6,
                   correlation_func_type='linear',
                   report_verbose=False)

experiment.run()
display(experiment.ratios)

,Batch,x1,x2,y
0,0.0,-8.630697,-5.509461,-15.177179
1,0.0,2.429707,4.461679,-3.034724
2,0.0,9.223992,-0.138533,-40.431662
3,0.0,-0.581540,9.872013,-10.784934
4,0.0,-4.725866,-2.768824,-6.485489
...,...,...,...,...
95,10.0,9.500722,9.179713,-52.924842
96,10.0,-5.411422,-1.461279,-19.625419
97,10.0,-6.973400,-7.820130,-42.202588
98,10.0,-5.802452,4.574697,-29.793642


,x1,x2,y
0,-8.630697,-5.509461,-15.177179
1,2.429707,4.461679,-3.034724
2,9.223992,-0.138533,-40.431662
3,-0.581540,9.872013,-10.784934
4,-4.725866,-2.768824,-6.485489
...,...,...,...
95,9.500722,9.179713,-52.924842
96,-5.411422,-1.461279,-19.625419
97,-6.973400,-7.820130,-42.202588
98,-5.802452,4.574697,-29.793642


,x1,x2
0.1,0.950888,0.049112


In [148]:
DIM = 5
LOWER = -10
UPPER = 10

# Initialize the Levy function
fun = Levy(dim=DIM, negate=True)
fun.bounds[0, :].fill_(LOWER)  # Bounds of the function
fun.bounds[1, :].fill_(UPPER)

def wrapped_levy_function(x):
    tensor_input = torch.tensor(x.values, dtype=torch.float32)
    
    if tensor_input.dim() == 1:
        tensor_input = tensor_input.unsqueeze(0)
    
    result = fun(tensor_input)
    
    return result.item()


df = pd.DataFrame([[1, 1]], columns=['x1', 'x2'])
output = wrapped_levy_function(df)
print(output)

-7.64274186065882e-15


In [154]:
import numpy as np
import pandas as pd
from varstool import VARS, Model

my_parameters = { 'x1': [ -10, 10 ], 
                  'x2': [ -10, 10 ], }


levy_model = Model(wrapped_levy_function)

# Initialize the VARS object
experiment_1 = VARS(parameters     = my_parameters,
                    num_stars      = 100,
                    delta_h        = 0.1,
                    ivars_scales   = (0.1, 0.3, 0.5),
                    sampler        = 'sobol_seq',
                    seed           = 123456789,
                    model          = levy_model,
                    bootstrap_flag = True,
                    bootstrap_size = 100,
                    bootstrap_ci   = 0.9,
                    report_verbose = True,
                    )

# Run the analysis
experiment_1.run_online()

bootstrapping: 100%|██████████| 100/100 [00:17<00:00,  5.70it/s]
/Users/dylanschubert/anaconda3/envs/msc_project/lib/python3.12/site-packages/varstool/sensitivity_analysis/vars_funcs.py:989: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  result_bs_sobol_ranking[param].eq(st_factor_ranking[param][0]).sum() / bootstrap_size)


In [155]:
import matplotlib.pyplot as plt
cols = experiment_1.parameters.keys()
df = experiment_1.ivars[cols]

row_sums = df.sum(axis=1)

normalized_df = df.div(row_sums, axis=0)
display(normalized_df)

,x1,x2
0.1,0.985902,0.014098
0.3,0.962494,0.037506
0.5,0.953577,0.046423
